<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/tools/DistillationNRTL_EtOH_iPrOH_H2O_che5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.2 MB/s eta 0:00:00


In [2]:
from importnb import Notebook
with Notebook():
    from che5 import DotDict, Range, RangeArray, Comp, CompArray, d2nt, Props

import numpy as np
from scipy.optimize import root,minimize
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True)

In [3]:
p=Props(['Ethanol','Isopropanol', 'Water'])

In [4]:
d=DotDict()

d.P = 1e5
d.N = 15
d.NC = p.N_comps
d.F= 100.
d.Fz = np.array([0.1,0.1, 0.8])
d.FT =320.
d.D = 0.15*d.F
d.B = d.F - d.D

d.F_stage = d.N // 2

d.reflux_ratio = 10


In [5]:
d.bp = jnp.squeeze(p.Tb(d.P))
bp_scaled = (d.bp - np.mean(d.bp))/1.2 + np.mean(d.bp)
bp_scaled_minmax = np.min(bp_scaled), np.max(bp_scaled)
bp_minmax = np.min(d.bp), np.max(d.bp)

d.T = RangeArray(np.linspace(*bp_scaled_minmax,d.N), *bp_minmax)
d.TC = Range(bp_scaled[-1], *bp_minmax)

Lguess = np.repeat(d.D*(d.reflux_ratio),d.N)
Lguess[d.F_stage:]+=d.F

d.L = RangeArray(Lguess, 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*(d.reflux_ratio+1),d.N), 0., 2*d.F*d.reflux_ratio)

QBGuess = d.D * (d.reflux_ratio+1)* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )
d.QC = Range(QBGuess, 0., 2*QBGuess)

d.Lx = CompArray(np.tile(d.Fz,(d.N,1)))
d.Vy = CompArray(np.tile(d.Fz,(d.N,1)))



In [14]:
x0, x2nt, wrap, x2unk, const   = d2nt(d)

In [28]:
r = DotDict()

r.MB = np.zeros((d.N,d.NC))
r.EB = np.zeros(d.N)
zeros = np.zeros((1,p.N_comps))
def eqs(d):
    r.EQUIL = d.Lx * p.NRTL_gamma(d.Lx,d.T)* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB[:] = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage.astype(int)-1] += d.F*d.Fz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]* p.NRTL_gamma(d.Vy[0],d.TC) * p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB[:] = p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:])
    r.EB[d.F_stage-1] += p.Hl(d.F*d.Fz, d.FT)
    r.EB[-1]+=d.QB

    r.CONDENSER = np.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r


In [30]:

def eqs_jax(d):
    zeros = jnp.zeros((1,p.N_comps))
    r = DotDict()
    r.EQUIL = d.Lx * p.NRTL_gamma(d.Lx,d.T)* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = jnp.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = jnp.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = jnp.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB=r.MB.at[d.F_stage.astype(int)-1].add(d.F*d.Fz)

    r.TCBP = jnp.atleast_1d(jnp.sum(d.Vy[0]* p.NRTL_gamma(d.Vy[0],d.TC) * p.Pvap(d.TC)) - d.P)

    r.T = jnp.insert(d.T,0,d.TC)
    r.EB = p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:])
    r.EB=r.EB.at[d.F_stage.astype(int)-1].add(p.Hl(d.F*d.Fz, d.FT))
    r.EB=r.EB.at[-1].add(d.QB)

    r.CONDENSER = jnp.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = jnp.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return jnp.concatenate([jnp.ravel(r.EQUIL), jnp.ravel(r.MB), jnp.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r

In [31]:
eqs_wrapped = wrap(eqs_jax)
eqs_jit=jax.jit(eqs_wrapped)
jac=jax.jit(jax.jacobian(eqs_wrapped))

In [32]:
sol=minimize(lambda x: 0., x0, method='SLSQP', bounds=[(-15,15)]*len(x0), constraints=dict(type='eq',fun=eqs_jit,jac=jac))
# sol=root(eqs_jit,x0,jac=jac,method='lm')

In [33]:
eqs_jax(x2nt(sol.x))

(Array([-5.82076609e-11, -2.61934474e-10,  1.12777343e-10,  1.96450856e-09,
         1.95723260e-09,  1.55341695e-09, -2.11002771e-10,  4.14729584e-10,
        -1.38243195e-10, -2.03726813e-10, -3.20142135e-10,  0.00000000e+00,
         7.27595761e-12,  3.63797881e-11, -2.65572453e-10, -7.27595761e-11,
         3.12866177e-10, -7.27595761e-11, -1.30967237e-10, -1.60071068e-10,
        -1.45519152e-11,  7.63975549e-11,  7.27595761e-12, -1.81898940e-10,
         9.09494702e-11, -6.54836185e-11, -3.63797881e-11, -1.16415322e-10,
        -1.67347025e-10, -2.32830644e-10, -4.72937245e-11, -4.07453626e-10,
        -2.61934474e-10,  1.09139364e-10, -9.45874490e-11,  1.01863407e-10,
        -9.45874490e-11, -6.33008312e-10, -2.18278728e-10, -3.63797881e-11,
        -5.45696821e-10, -1.16415322e-10,  6.18456397e-11, -2.14640750e-10,
         2.83762347e-10, -1.42108547e-14, -7.10542736e-15,  7.10542736e-15,
        -7.10542736e-15,  0.00000000e+00,  0.00000000e+00,  2.84217094e-14,
         0.0

In [ ]:
x2nt(sol.x)._asdict()